In [1]:
import numpy as np
import tensorflow as tf
from load_dataset_VGG16 import load_dataset
import matplotlib.pyplot as plt
import time

In [2]:
x_train,t_train = load_dataset('./train_dataset_AlexNet',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)
x_test,t_test = load_dataset('./test_dataset_AlexNet',convert_type='RGB',flatten=True,normalize=True,one_hot_label=True)

Load label : Done!
Load img : Done!
Load label : Done!
Load img : Done!


In [5]:
X = tf.placeholder(tf.float32, shape=[None, 224*224*3],name="input")
t = tf.placeholder(tf.float32, shape=[None, 4])
keep_prob = tf.placeholder(tf.float32,name="keep_prob")
### 入力層
input_layer = tf.reshape(X, [-1,224,224,3])

In [6]:
### 畳込み層、プーリング層1
h_conv1 = tf.layers.conv2d(inputs=input_layer,filters=96,kernel_size=[11, 11],strides=(4,4),padding='same',activation=tf.nn.relu)
h_norm1 = tf.nn.local_response_normalization(h_conv1,depth_radius=2,alpha=2e-5,beta=0.75)
h_pool1 = tf.layers.max_pooling2d(inputs=h_norm1,pool_size=(3,3),strides=(2,2),padding='same')

In [7]:
### 畳込み層、プーリング層2
h_conv2 = tf.layers.conv2d(inputs=h_pool1,filters=256,kernel_size=[5, 5],padding='same',activation=tf.nn.relu)
h_norm2 = tf.nn.local_response_normalization(h_conv2,depth_radius=2,alpha=2e-5,beta=0.75)
h_pool2 = tf.layers.max_pooling2d(inputs=h_norm2,pool_size=(3,3),strides=(2,2),padding='same')
print(h_norm2)
print(h_pool2)

Tensor("LRN_1:0", shape=(?, 28, 28, 256), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 14, 14, 256), dtype=float32)


In [8]:
### 畳込み層、プーリング層3
h_conv3_1 = tf.layers.conv2d(inputs=h_pool2,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_2 = tf.layers.conv2d(inputs=h_conv3_1,filters=384,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)
h_conv3_3 = tf.layers.conv2d(inputs=h_conv3_2,filters=256,kernel_size=[3, 3],padding='same',activation=tf.nn.relu)

h_pool3 = tf.layers.max_pooling2d(inputs=h_conv3_3,pool_size=(3,3),strides=(2,2),padding='same')

print(h_conv3_3)
print(h_pool3)

Tensor("conv2d_5/Relu:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 7, 7, 256), dtype=float32)


In [9]:
### 全結合層1

stddev = np.sqrt(1.0 / 7*7*256)
h_W_fc1 = tf.Variable(tf.truncated_normal([7*7*256,4096], stddev=stddev)) 
h_b_fc1 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_pool3_flat = tf.reshape(h_pool3, [-1, 7*7*256])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, h_W_fc1) + h_b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
### 全結合層2

stddev = np.sqrt(1.0 / 4096)
h_W_fc2 = tf.Variable(tf.truncated_normal([4096,4096], stddev=stddev)) 
h_b_fc2 = tf.Variable(tf.constant(0.1, shape=[4096]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, h_W_fc2) + h_b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [11]:
### 出力層
stddev = np.sqrt(2.0 / 4096)
W_fc3 = tf.Variable(tf.truncated_normal([4096,4], stddev=stddev))
b_fc3 = tf.Variable(tf.constant(0.1, shape=[4]))
y_conv = tf.nn.xw_plus_b(h_fc2_drop,W_fc3,b_fc3,name="output")

In [12]:
### 損失
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y_conv))

### 学習op
optimizer = tf.train.AdamOptimizer(1e-6)
train_step = optimizer.minimize(loss)

### モデルの評価
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(t,1))
# 精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 100
start = time.time()
for i in range(500):
    batch_mask = np.random.choice(x_train.shape[0], batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 10 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_train, t: t_train,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test, t: t_test,keep_prob:1.0})
        print "[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc)
        print("Time : %f" % (time.time() - start))
        
saver.save(sess, "./RoadSign-Alexnet-ckpt/roadsign_alex")

[Train] step: 0, loss: 21.421619, acc: 0.296047, [Test] acc : 0.270270
Time : 19.159147
[Train] step: 10, loss: 14.798828, acc: 0.384357, [Test] acc : 0.378378
Time : 26.510859
[Train] step: 20, loss: 11.714077, acc: 0.387721, [Test] acc : 0.405405
Time : 33.499298
[Train] step: 30, loss: 10.144858, acc: 0.386880, [Test] acc : 0.405405
Time : 40.614627
[Train] step: 40, loss: 8.788651, acc: 0.412111, [Test] acc : 0.351351
Time : 47.799222
[Train] step: 50, loss: 8.689365, acc: 0.428091, [Test] acc : 0.405405
Time : 54.866660
[Train] step: 60, loss: 8.642957, acc: 0.439024, [Test] acc : 0.432432
Time : 61.913993
[Train] step: 70, loss: 7.969517, acc: 0.444912, [Test] acc : 0.378378
Time : 69.240307
[Train] step: 80, loss: 8.021721, acc: 0.449117, [Test] acc : 0.378378
Time : 76.563772
[Train] step: 90, loss: 6.899406, acc: 0.456686, [Test] acc : 0.378378
Time : 83.869706
[Train] step: 100, loss: 6.428270, acc: 0.460891, [Test] acc : 0.351351
Time : 91.168086
[Train] step: 110, loss: 5.8

'./RoadSign-Alexnet-ckpt/roadsign_alex'

In [12]:
ckpt = tf.train.get_checkpoint_state('./RoadSign-Alexnet-ckpt')
if ckpt:
    # checkpointファイルから最後に保存したモデルへのパスを取得する
    last_model = ckpt.model_checkpoint_path
    print("load {0}".format(last_model))
    # 学習済みモデルを読み込む
    saver.restore(sess, last_model)

batch_size = 100
start = time.time()
for i in range(700):
    batch_mask = np.random.choice(x_train.shape[0], batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    sess.run(train_step,feed_dict={X: x_batch, t: t_batch,keep_prob:0.5})
    if i % 10 == 0:
        train_acc, train_loss = sess.run([accuracy,loss], feed_dict={X: x_train, t: t_train,keep_prob:1.0})
        test_acc = sess.run(accuracy, feed_dict={X: x_test, t: t_test,keep_prob:1.0})
        print "[Train] step: %d, loss: %f, acc: %f, [Test] acc : %f" % (i, train_loss, train_acc,test_acc)
        print("Time : %f" % (time.time() - start))
        
saver.save(sess, "./RoadSign-Alexnet-ckpt2/roadsign_alex")

load ./RoadSign-Alexnet-ckpt/roadsign_alex
INFO:tensorflow:Restoring parameters from ./RoadSign-Alexnet-ckpt/roadsign_alex
[Train] step: 0, loss: 1.581534, acc: 0.677040, [Test] acc : 0.702703
Time : 2.500137
[Train] step: 10, loss: 1.556070, acc: 0.677881, [Test] acc : 0.675676
Time : 9.678245
[Train] step: 20, loss: 1.649938, acc: 0.673675, [Test] acc : 0.702703
Time : 16.767586
[Train] step: 30, loss: 1.611284, acc: 0.677040, [Test] acc : 0.702703
Time : 23.955387
[Train] step: 40, loss: 1.468298, acc: 0.688814, [Test] acc : 0.702703
Time : 31.063954
[Train] step: 50, loss: 1.416686, acc: 0.693019, [Test] acc : 0.702703
Time : 38.321554
[Train] step: 60, loss: 1.362910, acc: 0.698066, [Test] acc : 0.702703
Time : 45.500528
[Train] step: 70, loss: 1.415463, acc: 0.699748, [Test] acc : 0.702703
Time : 52.580843
[Train] step: 80, loss: 1.516504, acc: 0.688814, [Test] acc : 0.702703
Time : 59.697216
[Train] step: 90, loss: 1.561564, acc: 0.692178, [Test] acc : 0.702703
Time : 66.856037


'./RoadSign-Alexnet-ckpt2/roadsign_alex'